# Statistical Analysis of ERP Data

In the previous lesson we visualized the difference waveform plot.<br>
Based on this plot, we can infer a significant difference between experimental conditions (Violation–Control).

In the present lesson, we will proceed to the statistical analysis of ERP data in order to check whether the Violation–Control difference is statistically significant.

For this purpose, we will perform a **one-sample t-test** and compute a **p-value**.

A one-sample t-test statistically determines if the mean of a single sample significantly differs from a known or hypothesized population mean (a specific constant value).

The p-value indicates the probability that the observed result will occur if the null hypothesis is true.

The null hypothesis assumes that there is no difference between two or more groups with respect to a characteristic.

We test if the observed difference is due to chance or a real effect, by checking if the p-value is below a chosen significance level (e.g., 0.05):
* a small p-value (e.g., < 0.05) suggests a statistically significant difference, meaning you reject the null hypothesis
* a large p-value (e.g., > 0.05) means you fail to reject it, indicating the observed difference could be due to chance.

## Install MNE Library

In [3]:
# install MNE library
!pip install -q mne

## Download Data Files

The present study involves 26 data files.<br>
The list of the locations of these files is stored in a text file named `sentence_n400_files_list.txt`.

First, we download this text file, and then we download the data files using the `wget` utility.

In [4]:
# download text file containing the list of data files
!wget -q https://datascience.faseela.ma/wp-content/uploads/data-science/sentence_n400_files_list.txt

# download data files
!wget -q -i https://datascience.faseela.ma/wp-content/uploads/data-science/sentence_n400_files_list.txt

## Import Libraries

### SciPy library

t-test is implemented in the SciPy library.

We will use the **`scipy.stats`** module from SciPy.

In [ ]:
import mne
mne.set_log_level('error')

import matplotlib.pyplot as plt
import numpy as np
import glob

from scipy import stats

## Define Parameters

We define a list of experimental conditions.

In [ ]:
# define experimental conditions
conditions = ['Control', 'Violation']

## Set a List of the Data Files

We use the Python **`glob`** module which allows to find pathnames matching a specified pattern, and returns a list of path names matching that pattern.

In [ ]:
# create a list of the data files
data_files = glob.glob('sentence_n400_p*-ave.fif' )

# print list of the data files
data_files

['sentence_n400_p12-ave.fif',
 'sentence_n400_p01-ave.fif',
 'sentence_n400_p22-ave.fif',
 'sentence_n400_p13-ave.fif',
 'sentence_n400_p10-ave.fif',
 'sentence_n400_p02-ave.fif',
 'sentence_n400_p17-ave.fif',
 'sentence_n400_p23-ave.fif',
 'sentence_n400_p24-ave.fif',
 'sentence_n400_p09-ave.fif',
 'sentence_n400_p06-ave.fif',
 'sentence_n400_p04-ave.fif',
 'sentence_n400_p14-ave.fif',
 'sentence_n400_p25-ave.fif',
 'sentence_n400_p03-ave.fif',
 'sentence_n400_p08-ave.fif',
 'sentence_n400_p07-ave.fif',
 'sentence_n400_p20-ave.fif',
 'sentence_n400_p18-ave.fif',
 'sentence_n400_p26-ave.fif',
 'sentence_n400_p21-ave.fif',
 'sentence_n400_p19-ave.fif',
 'sentence_n400_p15-ave.fif',
 'sentence_n400_p16-ave.fif',
 'sentence_n400_p05-ave.fif',
 'sentence_n400_p11-ave.fif']

## Read the Data Files

We store the data files as a dictionary, where:
* each key is a condition label,
* each value is a list of **`Evoked`** objects (the data from that condition, with each participant’s **`Evoked`** object as a list item).

We use the **`enumerate()`** function to loop over conditions and build our dictionary of **`Evoked`** objects.

We use the index from the **`enumerate()`** function to specify which condition (list item) we read from each participant’s data file.

We use list comprehension to build the list of **`Evoked`** objects for each condition.

In [ ]:
# create dictionary for Evoked objects
evokeds = {}

for idx, c in enumerate(conditions):
    evokeds[c] = [mne.read_evokeds(d)[idx].set_montage('easycap-M1') for d in data_files]

## Difference Waveforms

We create difference waveforms to more easily visualize the difference between conditions, and compare it to zero (i.e., no difference between conditions).

In order to get CIs that reflect the variance across participants, we need to compute the Violation-Control difference separately for each participant.

We use the **`mne.combine_evoked()`** function which merges multiple **`Evoked`** data objects by calculating their weighted sum.

We set the values of weights as `1` for Violation and `-1` for Control.

We put this function in a list comprehension to loop over participants.

In [ ]:
# create a list of difference waveform
diff_waves = [mne.combine_evoked([evokeds['Violation'][subj],
                                  evokeds['Control'][subj]
                                 ],
                                 weights=[1, -1]
                                 )
              for subj in range(len(data_files))
              ]

## t-test

To test whether an ERP effect is significant we perform a one-sample t-test between a pair of conditions, on the ERP data averaged over a time period of interest and at one or a few electrodes.

In the present study, we predicted an N400, which we knew from previous studies with similar stimuli would likely be largest between 400–600 ms, at midline channels Cz, CPz, and Pz.

We will use the **`scipy.stats.ttest_1samp()`** function from SciPy’s stats module to perform a one-sample t-test.

We use as input the Violation-Control differences for each participant (**`diff_waves`**), since if they are significantly different from zero, that will be evidence for a difference in amplitude between these two conditions.

We first compute the average over the 400–600 ms time window, for each participant, at the channels of interest, and store these in a NumPy array on which we then perform the t-test.

The rows in the array correspond to participants, and the columns to channels.

To create this Numpy array, we perform the following steps:

* **Step 1:** We define the time window and channels of interest as variables.<br>
We use tuples to indicate that we would not want these values changed after they’re defined.

* **Step 2:** We create a list named **`evoked_data_list`**.<br>
Each item in this list is a 2D Numpy array with shape (number of channels, number of time points).<br>
This array stores the data of an **`Evoked`** object.<br>
We create this list as follows:
   * We use a list comprehension to loop through the items in the list **`diff_waves`**. Each item is an **`Evoked`** object.
   * We use the **`evoked.get_data()`** method to extract data from the **`Evoked`** object.<br>
   This method returns data as 2D Numpy array with shape (number of channels, number of time points).
   * We set some keywords arguments of the **`evoked.get_dat()`** method:
      * We use the **`picks`** keyword argument to specify which electrode(s) we want. Here we pass the **`roi tuple`**.
      * We use the **`tmin`** and **`tmax`** keywords arguments to specify the time range over which we want to extract the ERP data. This is the tuple **`time_win`**.
   * Result: each item in the list is a 2D Numpy array storing evoked data of a participant.

* **Step 3:** We create a list named **`evoked_mean_list`**.<br>
Each item in the list is a 1D Numpy array with shape (number of channels).<br> Each element in the array is the mean of a channel.<br>
We create this list as follows:
   * We use a list comprehension to loop through the items in the list **`evoked_data_list`**. Each item is a Numpy array.
   * We use the **`np.mean()`** function to compute the average of each column in the Numpy array.<br>
   We set the **`axis=1`** keyword argument to instructs the function to average over columns (axis 1 corresponds to time points) and not rows (axis 0 corresponds to electrodes).

* **Step 4:**  We create a Numpy array named **`y`** from the list **`evoked_mean_list`**, using the **`np.array()`** function, which converts the list to a 2D Numpy array with 26 rows (participants) and 3 columns (channels).

In [ ]:
# set time window
time_win = (.400, .600)

# set region of interest
roi = ('Cz', 'CPz', 'Pz')

# create a list in which each item is a 2D numpy array storing
# a participant evoked data stored in diff_waves list
evoked_data_list = [item.get_data(picks=roi,
                               tmin=time_win[0],
                               tmax=time_win[1]
                               )
                   for item in diff_waves
                   ]

# create a list in which each item is a 1D numpy array
# each element in this array is the mean of columns of
# the numpy array which is an item in the list evoked_data_list
evoked_mean_list = [np.mean(item, axis=1) for item in evoked_data_list]

# create a 2D numpy array from the list evoked_mean_list
sample_data = np.array(evoked_mean_list)

# check shape of result
sample_data.shape

(26, 3)

**Note:** in the code above, we have unpacked the steps so one can understand the process.<br>
We can pack this code as follows:

In [ ]:
sample_data = np.array([np.mean(e.get_data(picks=roi,
                                 tmin=time_win[0],
                                 tmax=time_win[1]
                                 ),
                      axis=1)
              for e in diff_waves
              ]
             )

### Perform the t-test

We use the **`scipy.stats.ttest_1samp()`** function which performs a one-sample t-test to determine if the mean of a sample is statistically different from a given population mean.

It returns the calculated **t-statistic** and the **p-value**.

It produces NumPy arrays for its outputs which here we assign to variables we name **`t_statistic`** and **`p_value`**.

As this function returns arrays, we print the first element of each array.

We call this function with two arguments as **`ttest_1samp(a, popmean)`**:
* **`a`**: array_like, sample observations.
* **`popmean`**: float, expected value (mean) under the null hypothesis.

In [ ]:
# set expected value (mean) under the null hypothesis
null_hypothesis_mean = 0

# perform t-test
t_statistic, p_value = stats.ttest_1samp(sample_data, null_hypothesis_mean)

# print results
print('t-statistic = ', str(round(t_statistic[0], 2)))
print('p-value = ', str(round(p_value[0], 4)))

t-statistic =  -3.23
p-value =  0.0035


The p-value is lower than 0.05, so we can reject the null hypothesis of no difference between the conditions, and conclude that the difference between conditions is statistically significant.